In [1]:
from os import environ as ENV
from dotenv import load_dotenv
import pandas as pd
import altair as alt
from load import get_redshift_conn
from report import yesterday_data_to_dataframe

In [2]:
load_dotenv()
conn = get_redshift_conn(ENV)
df = yesterday_data_to_dataframe(conn)
df.info()
conn.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   transaction_id  495 non-null    int64         
 1   truck_id        495 non-null    int64         
 2   payment_type    495 non-null    object        
 3   total           495 non-null    float64       
 4   at              495 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 19.5+ KB


In [3]:
total_transactions = df["transaction_id"].count()

print(f"Total transactions: {total_transactions}")

Total transactions: 495


In [4]:
transactions_per_truck = df.groupby(df['truck_id']).count().sort_values(
    'total', ascending=False).drop(columns=['at', 'payment_type', 'transaction_id']).reset_index()

alt.Chart(transactions_per_truck).mark_bar().encode(
    x=alt.X("truck_id:N", title=None),
    y=alt.Y("total:Q", title="Total transactions"),
    color=alt.Color("truck_id:N", title="Truck ID", scale=alt.Scale(domain=[1, 2, 3, 4, 5, 6], range=['#E4572E',  '#A8C686', '#669BBC', '#D4878D', '#29335C', '#F3A712'])))

alt.Chart(...)

In [5]:
total_transaction_value = df.groupby(df['truck_id'])["total"].sum().reset_index().sort_values('total', ascending=False)

alt.Chart(total_transaction_value).mark_bar().encode(
    x=alt.X("truck_id:N", title=None),
    y=alt.Y("total:Q", title="Total transaction value (£)"),
    color=alt.Color("truck_id:N", title="Truck ID", scale=alt.Scale(domain=[1, 2, 3, 4, 5, 6], range=['#E4572E',  '#A8C686', '#669BBC', '#D4878D', '#29335C', '#F3A712'])))

alt.Chart(...)

In [13]:
total_payment_methods = df["payment_type"].value_counts().reset_index()
total_payment_methods["Proportion (%)"] = ((total_payment_methods["count"] / df["payment_type"].count()) * 100).round(2)

alt.Chart(total_payment_methods).mark_arc().encode(
    theta="Proportion (%)",
    color=alt.Color('payment_type', scale=alt.Scale(domain=['card', 'cash'], range=['#29335C', '#F3A712']), title="Payment method")
    )

alt.Chart(...)

In [7]:
payment_methods = df.groupby(df["truck_id"])["payment_type"].value_counts().reset_index()
payment_methods["Proportion (%)"] = ((payment_methods["count"] / df["payment_type"].count()) * 100).round(2)

title = alt.TitleParams("Payment methods by food_truck", anchor='middle')
alt.Chart(payment_methods, title=title).mark_bar().encode(
    x = alt.X('payment_type:O', title=None),
    y = alt.Y('count:Q', title="Number of Transactions"),
    column = 'truck_id:N',
    color = alt.Color('payment_type', scale=alt.Scale(domain=['card', 'cash'], range=['#29335C', '#F3A712'])))

alt.Chart(...)

In [8]:
avg_val_per_truck = df.groupby(df['truck_id'])["total"].mean().reset_index().round(2)
avg_val_per_truck.head(6).reset_index().round(2).drop(columns=['index'])

alt.Chart(avg_val_per_truck).mark_bar().encode(
    x=alt.X("truck_id:N", title=None),
    y=alt.Y("total:Q", title="Average transaction price"),
    color=alt.Color("truck_id:N", title="Truck ID", scale=alt.Scale(domain=[1, 2, 3, 4, 5, 6], range=['#E4572E',  '#A8C686', '#669BBC', '#D4878D', '#29335C', '#F3A712'])))

alt.Chart(...)

In [9]:
df["d_of_wk"] = [date_time.weekday() for date_time in df['at']]

by_day = df.groupby(df['d_of_wk'])[
    'd_of_wk'].value_counts().reset_index()

day_mapping = {0: "Monday", 1: "Tuesday", 2: "Wednesday",
                3: "Thursday", 4: "Friday", 5: "Saturday", 6: "Sunday"}

by_day["Weekday"] = [day_mapping[num]for num in by_day['d_of_wk']]

days_of_week = ['Monday', 'Tuesday', 'Wednesday',
                'Thursday', 'Friday', 'Saturday', 'Sunday']

by_day['Weekday'] = pd.Categorical(
    by_day['Weekday'], categories=days_of_week, ordered=True)

alt.Chart(by_day).mark_bar().encode(
    x=alt.X("Weekday:N", sort=days_of_week, title=None),
    y=alt.Y("count:Q", title="Transactions per day"),
    color=alt.Color("Weekday:N", sort=days_of_week, scale=alt.Scale(domain=days_of_week, range=[
                    '#E4572E',  '#A8C686', '#669BBC', '#D4878D', "#432534", '#F3A712', '#29335C']))
    ).properties(title="Total transactions per day of the week.")

alt.Chart(...)